In [7]:
import os
import json
import requests
import pandas as pd
import sqlite3 as db
from pprint import pprint
from dotenv import load_dotenv

In [5]:
load_dotenv()
api_key = os.getenv('TRIPADVISOR_KEY')

In [6]:
location_no = 4870298
language = 'en'

url = f"https://api.content.tripadvisor.com/api/v1/location/{location_no}/reviews?key={api_key}&language={language}"
headers = {"accept": "application/json"}
response = requests.get(url, headers=headers)
print(response.text)

{
  "data": [
    {
      "id": "888265115",
      "lang": "en",
      "location_id": "4870298",
      "published_date": "2023-04-29T12:56:17-0400",
      "rating": 5,
      "helpful_votes": "0",
      "rating_image_url": "https://www.tripadvisor.com/img/cdsi/img2/ratings/traveler/s5.0-66827-5.svg",
      "url": "https://www.tripadvisor.com/ShowUserReviews-g60724-d4870298-r888265115-Pack_Creek_Campground-Moab_Utah.html?m=66827#review888265115",
      "travel_date": "2023-04",
      "text": "Well laid out, quiet campground for tents and campers,  in a residential area off the main road several blocks.  Good communication,  smooth after hours check-in. Has clean bathrooms with hot showers,  a playground,  and a group covered picnic area.  Has longterm trailers and vacation camping.  Tent sites are separated by fences.  Has a dumping station.  ",
      "title": "Great place to camp when in Moab.",
      "user": {
        "username": "kht53",
        "user_location": {
          "name": "C

In [15]:
reviews = []
for row in response.json()['data']:
    review = {
        'id': int(row['id']),
        'pub_date': row['published_date'],
        'title': row['title'],
        'review_text': row['text'],
        'rating': row['rating']
    }
    reviews.append(review)

df = pd.DataFrame(reviews)
df

,id,pub_date,title,review_text,rating
0,888265115,2023-04-29T12:56:17-0400,Great place to camp when in Moab.,"Well laid out, quiet campground for tents and ...",5
1,886090483,2023-04-13T10:53:46-0400,"nice, grown-up place to camp. Owner on site fo...",Friendly and helpful owner & staff. Beautiful ...,5
2,842850829,2022-06-14T09:27:49-0400,First impressions are not always right,When you pull up - don't be like me and have a...,5
3,840976834,2022-06-01T07:33:05-0400,Pack Creek Campground,Was there May 2022. Not a nice campground. Nob...,1
4,840573478,2022-05-29T13:36:07-0400,Great stay!,The best part of our stay at Pack Creek Campgr...,5


In [46]:
values = [str(tuple(row)) for row in df.values]
values = ',\n'.join(values) + ';'


sql = f'''
INSERT INTO reviews (id, pub_date, title, review_text, rating) VALUES
{values}
'''


In [48]:
db_path = os.path.normpath(os.path.join(os.getcwd(), '..', 'data', 'reviews.sqlite'))
conn = db.connect(db_path)
cursor = conn.cursor()
cursor.execute(sql)
conn.commit()


In [49]:
cursor.execute("SELECT * FROM reviews")
data = cursor.fetchall()
for row in data:
    print(row)

(888265115, '2023-04-29T12:56:17-0400', 'Great place to camp when in Moab.', 'Well laid out, quiet campground for tents and campers,  in a residential area off the main road several blocks.  Good communication,  smooth after hours check-in. Has clean bathrooms with hot showers,  a playground,  and a group covered picnic area.  Has longterm trailers and vacation camping.  Tent sites are separated by fences.  Has a dumping station.  ', 5)
(886090483, '2023-04-13T10:53:46-0400', 'nice, grown-up place to camp. Owner on site for any questions.', 'Friendly and helpful owner & staff. Beautiful and quiet location, right next to a river. You can camp at ground level or lower at the river so you hear the quietly babbling brook at night. Clean and daily maintained bathrooms and showers. laundry, large outdoor dish sink. Water at every 2 camp sites. about 1/2 mile out of town. Oh, and a great price!', 5)
(842850829, '2022-06-14T09:27:49-0400', 'First impressions are not always right', "When you pu

In [50]:
conn.close()